# Normalize DataFrames
Vamos a normalizar tanto los datos de entrenamiento 'train' como los de 'test'.

| | original data | normalized data |
|-------------------|---------------|-----------------|
| training data | dataset_train.csv | normal_train.csv |
| test data | dataset_test.csv | normal_test.csv 

Diferencias entre los datos de entrenamiento y de test originales:
- Los datos de entrenamiento tienen 1600 registros y los de test 400 registros
- Los datos de test tienen vacía la columna 'Hogwarts House' que es la columa objetivo de la clasificación.

Trabajaremos con una versión reducida (lite5) que contiene:
- Las características base (Best Hand, Age)
- 5 asignaturas principales: Herbology, Defense Against the Dark Arts, Potions, Charms, y Flying
- para los datos de entrenamiento también se incluyen las variables dummy de Hogwarts House 

In [1]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
from datetime import datetime

# Definición de las características que vamos a mantener
BASE_FEATURES = ['Best Hand', 'Age']
LITE5_COURSES = ['Herbology', 'Defense Against the Dark Arts',
                 'Potions', 'Charms', 'Flying'
]

## Lectura y preparación inicial de los datos

Leemos los datasets originales y calculamos la edad a partir de la columna Birthday.

In [2]:
# Lectura de los datasets
file_train = '../datasets/dataset_train.csv'
file_test = '../datasets/dataset_test.csv'

df_train = pd.read_csv(file_train, index_col=0)
df_test = pd.read_csv(file_test, index_col=0)

### Cálculo de la edad

In [3]:
# Convertimos Birthday a datetime y calculamos Age para ambos datasets
def calculate_age(df):
    df['Birthday'] = pd.to_datetime(df['Birthday'])
    reference_date = df['Birthday'].max()
    df['Age'] = (reference_date - df['Birthday']).dt.days / 365.25
    return df

df_train = calculate_age(df_train)
df_test = calculate_age(df_test)

# Eliminamos las columnas que no necesitamos
columns_to_drop = ['First Name', 'Last Name', 'Birthday']
df_train = df_train.drop(columns=columns_to_drop)
df_test = df_test.drop(columns=columns_to_drop)

### Convertimos 'Best Hand' a valores numéricos

In [4]:
# Convertimos Best Hand a valores numéricos en ambos datasets
hand_mapping = {'Left': 0, 'Right': 1}
df_train['Best Hand'] = df_train['Best Hand'].map(hand_mapping)
df_test['Best Hand'] = df_test['Best Hand'].map(hand_mapping)

# Convertimos Best Hand a float en ambos datasets
df_train['Best Hand'] = df_train['Best Hand'].astype(float)
df_test['Best Hand'] = df_test['Best Hand'].astype(float)

# Verificamos la conversión
print("Valores únicos en Best Hand (train):", df_train['Best Hand'].unique())
print("Valores únicos en Best Hand (test):", df_test['Best Hand'].unique())

Valores únicos en Best Hand (train): [0. 1.]
Valores únicos en Best Hand (test): [1. 0.]


## Tratamiento específico para los datos de entrenamiento

Para los datos de entrenamiento necesitamos hacer one-hot encoding de la columna 'Hogwarts House'.

In [5]:
# One-hot encoding solo para los datos de entrenamiento
df_train = pd.get_dummies(df_train, columns=['Hogwarts House'], prefix='House', dtype=float)

In [6]:
# Actualizamos BASE_FEATURES para incluir las columnas dummy de House
HOUSE_FEATURES = ['House_Gryffindor', 'House_Hufflepuff', 'House_Ravenclaw', 'House_Slytherin']
BASE_FEATURES = BASE_FEATURES + HOUSE_FEATURES

## Selección de características y normalización

Seleccionamos solo las características de lite5 y normalizamos las columnas numéricas.

In [7]:
# Seleccionamos las columnas que queremos mantener
df_train = df_train[LITE5_COURSES + BASE_FEATURES]
df_test = df_test[LITE5_COURSES + ['Best Hand', 'Age']]  # Note que no incluimos HOUSE_FEATURES para test

### Eliminación de las filas con datos faltantes
Se eliminan solo las filas de las características con las que se trabaja.

In [8]:
# Filas de los DataFrame previos a la eliminación de filas faltantes
n_train = len(df_train)
n_test = len(df_test)

df_train = df_train.dropna()
df_test = df_test.dropna()

print(f"El DataFrame train ha pasado de {n_train} filas a {len(df_train)} filas. Se han eliminado {n_train - len(df_train)} registros con datos faltantes.")
print(f"El DataFrame test ha pasado de {n_test} filas a {len(df_test)} filas. Se han eliminado {n_test - len(df_test)} registros con datos faltantes.")

El DataFrame train ha pasado de 1600 filas a 1508 filas. Se han eliminado 92 registros con datos faltantes.
El DataFrame test ha pasado de 400 filas a 372 filas. Se han eliminado 28 registros con datos faltantes.


### Función de Normalización

In [9]:
# Función de normalización
def normalize(column):
    mean = column.mean()
    std = column.std()
    return (column - mean) / std

### Aplicación de la Normalización a ambos DataFrames

In [10]:
# Para el dataset de entrenamiento
train_columns_to_normalize = df_train.select_dtypes(include=['float64']).columns.tolist()
df_train[train_columns_to_normalize] = df_train[train_columns_to_normalize].apply(normalize)

# Para el dataset de test (que no incluye las columnas de House)
test_columns_to_normalize = df_test.select_dtypes(include=['float64']).columns.tolist()
df_test[test_columns_to_normalize] = df_test[test_columns_to_normalize].apply(normalize)

# Verificamos las columnas normalizadas
print("Columnas normalizadas en train:\n", train_columns_to_normalize)
print("\nColumnas normalizadas en test:\n", test_columns_to_normalize)

Columnas normalizadas en train:
 ['Herbology', 'Defense Against the Dark Arts', 'Potions', 'Charms', 'Flying', 'Best Hand', 'Age', 'House_Gryffindor', 'House_Hufflepuff', 'House_Ravenclaw', 'House_Slytherin']

Columnas normalizadas en test:
 ['Herbology', 'Defense Against the Dark Arts', 'Potions', 'Charms', 'Flying', 'Best Hand', 'Age']


In [11]:
# Guardado de los datasets normalizados
df_train.to_csv('../datasets/normal_train.csv')
df_test.to_csv('../datasets/normal_test.csv')

print("Dimensiones del dataset de entrenamiento:", df_train.shape)
print("Dimensiones del dataset de test:", df_test.shape)
print("\nColumnas en el dataset de entrenamiento:", df_train.columns.tolist())
print("\nColumnas en el dataset de test:", df_test.columns.tolist())

Dimensiones del dataset de entrenamiento: (1508, 11)
Dimensiones del dataset de test: (372, 7)

Columnas en el dataset de entrenamiento: ['Herbology', 'Defense Against the Dark Arts', 'Potions', 'Charms', 'Flying', 'Best Hand', 'Age', 'House_Gryffindor', 'House_Hufflepuff', 'House_Ravenclaw', 'House_Slytherin']

Columnas en el dataset de test: ['Herbology', 'Defense Against the Dark Arts', 'Potions', 'Charms', 'Flying', 'Best Hand', 'Age']
